In [4]:
# chapter3
# 3.1 단일 도메인 내의 이동
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re

random.seed(datetime.datetime.now())

def getLinks(articleUrl):
    html = urlopen('http://en.wikipedia.org{}'.format(articleUrl))
    bs = BeautifulSoup(html, 'html.parser')
    return bs.find('div', {'id':'bodyContent'}).findAll('a', href=re.compile('^(/wiki/)((?!:).)*$'))

links = getLinks('/wiki/Kevin_Bacon')
while len(links) > 0:
    newArticle = links[random.randint(0, len(links)-1)].attrs['href']
    print(newArticle)
    links=getLinks(newArticle)

C:\Users\bmc05\AppData\Local\Temp\ipykernel_7768\4169568480.py:9: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


/wiki/Mads_Mikkelsen
/wiki/Jean_Reno
/wiki/French_Kiss_(1995_film)
/wiki/Turner_Classic_Movies
/wiki/Walt_Disney_anthology_television_series
/wiki/Francis_Marion
/wiki/Campfire_story
/wiki/Cowboy
/wiki/Charro
/wiki/Rurales
/wiki/Gendarmerie
/wiki/Turkey
/wiki/Desertification
/wiki/Wildfire
/wiki/New_York_roof_hook
/wiki/Pike_pole
/wiki/Denailer
/wiki/Feller_buncher
/wiki/Splitting_maul
/wiki/Go-devil
/wiki/Steam_donkey
/wiki/Tsakat
/wiki/List_of_blade_materials
/wiki/Hitachi
/wiki/Power_tool
/wiki/Mechanism_(engineering)
/wiki/Cam_follower
/wiki/Deformation_(engineering)
/wiki/Ballistics
/wiki/Latin
/wiki/Hybrid_word
/wiki/Neurotransmitter
/wiki/Neoendorphins
/wiki/Hdl_(identifier)
/wiki/Unicode
/wiki/Warang_Citi
/wiki/Tanchangya_script
/wiki/Khudabadi_script
/wiki/New_York_Point
/wiki/Mountbatten_Brailler
/wiki/Braille_watch
/wiki/Telugu_Braille
/wiki/Sinhala_Braille
/wiki/National_Braille_Association
/wiki/Decapoint
/wiki/Persian_Braille
/wiki/Valentin_Ha%C3%BCy
/wiki/Mainland_Chines

KeyboardInterrupt: 

In [2]:
# 3.2 전체 사이트 크롤링
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org{}'.format(pageUrl))
    bs = BeautifulSoup(html, 'html.parser')
    for link in bs.findAll('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print(newPage)
                pages.add(newPage)
                getLinks(newPage)
                
getLinks('')

/wiki/Wikipedia
/wiki/Wikipedia:Protection_policy#semi
/wiki/Wikipedia:Requests_for_page_protection
/wiki/Wikipedia:Requests_for_page_protection/Administrator_instructions
/wiki/Wikipedia:Protection_policy
/wiki/Wikipedia:Lists_of_protected_pages
/wiki/Wikipedia:Protection_policy#Semi-protection
/wiki/Wikipedia:Perennial_proposals
/wiki/Wikipedia:Reliable_sources/Perennial_sources


KeyboardInterrupt: 

In [4]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

pages = set()
def getLinks(pageUrl):
    global pages
    html = urlopen('http://en.wikipedia.org'+pageUrl)
    bs = BeautifulSoup(html, 'html.parser')
    try:
        print(bs.h1.get_text())
        print(bs.find(id = 'mw-content-text').findAll('p')[0])
        print(bs.find(id = 'ca-edit').find('span').find('a').attrs['href'])
    except ArithmeticError:
        print('This page is missing something! No worries though!')
    for link in bs.findAll('a', href=re.compile('^(/wiki/)')):
        if 'href' in link.attrs:
            if link.attrs['href'] not in pages:
                newPage = link.attrs['href']
                print('------------\n'+newPage)
                pages.add(newPage)
                getLinks(newPage)
                

getLinks('')

Main Page
<p>The <b><a href="/wiki/Third_Punic_War" title="Third Punic War">Third Punic War</a></b> was the third and last of the <a href="/wiki/Punic_Wars" title="Punic Wars">Punic Wars</a> between <a href="/wiki/Ancient_Carthage" title="Ancient Carthage">Carthage</a> and <a href="/wiki/Roman_Republic" title="Roman Republic">Rome</a>, and lasted from 149 to 146 BC. The war was fought in what is now northern Tunisia. In 149 BC anti-Carthaginian factions in Rome manufactured a pretext for war. The Carthaginians surrendered all of their weapons, but the Romans pressed on to <a href="/wiki/Siege" title="Siege">besiege</a> the city of <a href="/wiki/Carthage" title="Carthage">Carthage</a> <i>(siege engine depicted)</i>. The Romans suffered repeated setbacks. A new Roman commander took over in 148 BC, and fared equally badly. <a href="/wiki/Scipio_Aemilianus" title="Scipio Aemilianus">Scipio Aemilianus</a> was appointed commander in Africa for 147 BC; he tightened the siege and prevented su

AttributeError: 'NoneType' object has no attribute 'find'

In [6]:
# 3.3 인터넷 크롤링
from urllib.request import urlopen, urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = set()
random.seed(datetime.datetime.now())

# 페이지에서 발견된 내부 링크를 모두 목록으로 만듭니다.
def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
    internalLinks=[]
    
    # /로 시작하는 링크를 모두 찾습니다.
    for link in bs.findAll('a', href=re.compile('^(/|.*' + includeUrl + ')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith('/')):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

# 페이지에서 발견된 외부 링크를 모두 목록으로 만듭니다.
def getExternalLinks(bs, excludeUrl):
    externalLinks=[]
    # 현재 URL을 포함하지 않으면서 http나 www로 시작하는 링크를 모두 찾습니다.
    for link in bs.findAll('a', href=re.compile('^(http|www)((?!' + excludeUrl +').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs, urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print('No external links, looking around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme,urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0,len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0,len(externalLinks)-1)]
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    followExternalOnly(externalLink)

followExternalOnly('http://oreilly.com')

C:\Users\bmc05\AppData\Local\Temp\ipykernel_3232\3805205455.py:9: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


Random external link is: https://oreillylearning.in/
Random external link is: https://www.oreilly.com/about/editorial_independence.html
Random external link is: https://www.oreilly.co.jp/index.shtml
Random external link is: http://makezine.jp/events
Random external link is: https://twitter.com/make_jp
No external links, looking around the site for one
Random external link is: https://twitterdatadash.com
Random external link is: https://twitter.com/privacy
Random external link is: https://www.twitterflightschool.com/sl/382652bc
Random external link is: https://twitter.com/en/privacy
Random external link is: https://twitter.ethicspointvp.com/custom/twitter/forms/data/form_data.asp
Random external link is: https://twitter.com/en/privacy
Random external link is: https://investor.twitterinc.com/


HTTPError: HTTP Error 403: Forbidden

In [7]:
# 사이트에서 찾은 외부 URL을 모두 리스트로 수집한다. 위 코드에서 추가한 버전
from urllib.request import urlopen, urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random

pages = set()
random.seed(datetime.datetime.now())

# 페이지에서 발견된 내부 링크를 모두 목록으로 만듭니다.
def getInternalLinks(bs, includeUrl):
    includeUrl = '{}://{}'.format(urlparse(includeUrl).scheme, urlparse(includeUrl).netloc)
    internalLinks=[]
    
    # /로 시작하는 링크를 모두 찾습니다.
    for link in bs.findAll('a', href=re.compile('^(/|.*' + includeUrl + ')')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in internalLinks:
                if(link.attrs['href'].startswith('/')):
                    internalLinks.append(includeUrl+link.attrs['href'])
                else:
                    internalLinks.append(link.attrs['href'])
    return internalLinks

# 페이지에서 발견된 외부 링크를 모두 목록으로 만듭니다.
def getExternalLinks(bs, excludeUrl):
    externalLinks=[]
    # 현재 URL을 포함하지 않으면서 http나 www로 시작하는 링크를 모두 찾습니다.
    for link in bs.findAll('a', href=re.compile('^(http|www)((?!' + excludeUrl +').)*$')):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in externalLinks:
                externalLinks.append(link.attrs['href'])
    return externalLinks

def getRandomExternalLink(startingPage):
    html = urlopen(startingPage)
    bs = BeautifulSoup(html, 'html.parser')
    externalLinks = getExternalLinks(bs, urlparse(startingPage).netloc)
    if len(externalLinks) == 0:
        print('No external links, looking around the site for one')
        domain = '{}://{}'.format(urlparse(startingPage).scheme,urlparse(startingPage).netloc)
        internalLinks = getInternalLinks(bs, domain)
        return getRandomExternalLink(internalLinks[random.randint(0,len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0,len(externalLinks)-1)]
def followExternalOnly(startingSite):
    externalLink = getRandomExternalLink(startingSite)
    print('Random external link is: {}'.format(externalLink))
    followExternalOnly(externalLink)

allExtLinks = set()
allIntLinks = set()

def getAllExternalLinks(siteUrl):
    html = urlopen(siteUrl)
    domain = '{}://{}'.format(urlparse(siteUrl).scheme, urlparse(siteUrl).netloc)
    bs = BeautifulSoup(html, 'html.parser')
    internalLinks = getInternalLinks(bs, domain)
    externalLinks = getExternalLinks(bs, domain)
    
    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print(link)
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            print(link)

allIntLinks.add('http://oreilly.com')
getAllExternalLinks('http://oreilly.com')

C:\Users\bmc05\AppData\Local\Temp\ipykernel_3232\3757412681.py:9: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.datetime.now())


https://www.oreilly.com
https://www.oreilly.com/member/login/
https://www.oreilly.com/online-learning/try-now.html
https://www.oreilly.com/online-learning/teams.html
https://www.oreilly.com/online-learning/government.html
https://www.oreilly.com/online-learning/academic.html
https://www.oreilly.com/online-learning/individuals.html
https://www.oreilly.com/online-learning/features.html
https://www.oreilly.com/online-learning/feature-certification.html
https://www.oreilly.com/online-learning/intro-interactive-learning.html
https://www.oreilly.com/online-learning/live-events.html
https://www.oreilly.com/online-learning/feature-answers.html
https://www.oreilly.com/online-learning/insights-dashboard.html
https://www.oreilly.com/radar/
https://www.oreilly.com/content-marketing-solutions.html
https://www.oreilly.com/ceros/345800-holiday-card-2022.html
https://learning.oreilly.com/start-trial/
https://www.oreilly.com/online-learning/testimonials.html
https://www.oreilly.com/online-learning/orei